# pip

In [ ]:
!pip install tensorflow_addons

# static

In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/train_images.csv'
TEST_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/test_images.csv'
SAMPLE_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/sample_submit.csv'

TRAIN_IMAGE_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/train_images'
TEST_IMAGE_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/test_images'

MODEL_PATH = '/content/drive/MyDrive/SANDBOX/DATASET/model.h5'

TRAIN_TEST_SPLIT = 0.1
IMAGE_SIZE = 640
BATCH_SIZE = 1
EPOCHS = 100

LABEL_NAMES = ['0 - best', '1 - good', '2 - processed', '3 - non-standard']

# function

## custom callback

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import accuracy_score, cohen_kappa_score
import numpy as np

class CustomCallback(Callback):
  
  def __init__(self, model, generator):
    self.model = model
    self.generator = generator
  
  def on_epoch_end(self, epoch, logs):
    acc = accuracy_score(np.argmax(self.model.predict(self.generator), axis=1), self.generator.classes)
    qwk = cohen_kappa_score(np.argmax(self.model.predict(self.generator), axis=1), self.generator.classes, weights='quadratic')
    print('Accuracy: {0:.3f}, QWK: {1:.3f}'.format(acc, qwk))

# load data

In [ ]:
import pandas as pd
# pd.set_option('display.max_colwidth', 100)

from keras.utils import to_categorical

df = pd.read_csv(TRAIN_PATH)
df_preds = pd.read_csv(TEST_PATH)

df['id'] = TRAIN_IMAGE_PATH+'/'+df['id']

df['class_num'] = df['class_num'].astype(str)

print('df shape: {0}, df_preds shape: {1}'.format(df.shape, df_preds.shape))

df shape: (1102, 2), df_preds shape: (1651, 1)


In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_VALIDATION_SPLIT = 0.2

df_train, df_test = train_test_split(df, test_size=TRAIN_TEST_SPLIT, random_state=2021)
df_train, df_val = train_test_split(df_train, test_size=TRAIN_VALIDATION_SPLIT, random_state=2021)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(featurewise_center=False,
                                   samplewise_center=False,
                                   featurewise_std_normalization=False,
                                   samplewise_std_normalization=False,
                                   zca_whitening=True,
                                   rotation_range=360,
                                   width_shift_range=1.0,
                                   height_shift_range=1.0,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=0.0,
                                   fill_mode='nearest',
                                   cval=0.0,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rescale=1./255.,
                                   preprocessing_function=None,
                                   data_format=None,)
# train_datagen = ImageDataGenerator(rescale=1./255.,)
val_datagen = ImageDataGenerator(rescale=1./255.,)
test_datagen = ImageDataGenerator(rescale=1./255.,)

train_generator = train_datagen.flow_from_dataframe(df_train, 
                                                    directory=TRAIN_IMAGE_PATH, 
                                                    x_col='id', 
                                                    y_col='class_num', 
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                    batch_size=BATCH_SIZE, 
                                                    class_mode='categorical')
val_generator = val_datagen.flow_from_dataframe(df_val, 
                                                directory=TRAIN_IMAGE_PATH, 
                                                x_col='id', 
                                                y_col='class_num', 
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                batch_size=BATCH_SIZE, 
                                                class_mode='categorical', 
                                                shuffle=False)
test_generator = test_datagen.flow_from_dataframe(df_test, 
                                                  directory=TRAIN_IMAGE_PATH, 
                                                  x_col='id', 
                                                  y_col='class_num', 
                                                  target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                  batch_size=BATCH_SIZE, 
                                                  class_mode='categorical', 
                                                  shuffle=False)

for data_batch, labels_batch in train_generator:
  print('data batch shape: ', data_batch.shape)
  print('labels batch shape: ', labels_batch.shape)
  break

Found 792 validated image filenames belonging to 4 classes.
Found 199 validated image filenames belonging to 4 classes.
Found 111 validated image filenames belonging to 4 classes.
data batch shape:  (1, 640, 640, 3)
labels batch shape:  (1, 4)


# create model

In [ ]:
from keras import models
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB5, EfficientNetB7
from keras.layers import Flatten, BatchNormalization, Dense, GlobalAveragePooling2D, Dropout

model = models.Sequential()

# conv_base = VGG16(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)
# conv_base = ResNet50(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)
# conv_base = EfficientNetB5(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)
conv_base = EfficientNetB7(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)

### freezing ###
# TARGET_LAYER = 'block5_conv2'        # VGG16
# TARGET_LAYER = 'conv5_block3_2_conv' # ResNet50

# conv_base.trainable = True
# set_trainable = False

# for layer in conv_base.layers:
#   if layer.name == TARGET_LAYER:
#     set_trainable = True
#   if set_trainable:
#     layer.trainable = True
#   else:
#     layer.trainable = False

for layer in conv_base.layers:
  layer.trainable = False

# model.add(Flatten())
# model.add(BatchNormalization())
# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dense(4, activation='softmax'))

model.add(conv_base)

model.add(GlobalAveragePooling2D())
# model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
# model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# conv_base.summary()

# fit

[WightedKappaLoss](https://www.tensorflow.org/addons/api_docs/python/tfa/losses/WeightedKappaLoss)

[CohenKappa](https://www.tensorflow.org/addons/api_docs/python/tfa/metrics/CohenKappa)



In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, SGD, Adagrad
from tensorflow_addons.losses import WeightedKappaLoss
from tensorflow_addons.metrics import CohenKappa
from tensorflow_addons.optimizers import RectifiedAdam
import time

LEARNING_RATE_WARMUP = 1e-3
EPOCHS_WARMUP = 100
LEARNING_RATE = 1e-5

print('### START WARMUP ###')

# MONITOR_EARLYSTOPPING = 'val_cohen_kappa'
# MONITOR_MODELCHECKPOINT = 'val_cohen_kappa'
MONITOR_EARLYSTOPPING = 'val_loss'
MONITOR_MODELCHECKPOINT = 'val_loss'

callbacks_list = [# EarlyStopping(monitor=MONITOR_EARLYSTOPPING, patience=5, mode='min',),
                  ReduceLROnPlateau(monitor='val_loss', mode='min', patience=3, factor=0.5, min_lr=1e-6, verbose=1),
                  CustomCallback(model, val_generator),
                  ModelCheckpoint(filepath=MODEL_PATH, monitor=MONITOR_MODELCHECKPOINT, save_best_only=True, mode='min',),]

model.compile(optimizer=Adam(lr=LEARNING_RATE_WARMUP),
              # optimizer=RectifiedAdam(total_steps=10000, warmup_proportion=0.1, min_lr=1e-5),
              # optimizer=Adagrad(lr=LEARNING_RATE_WARMUP),
              # optimizer=SGD(lr=1e-3, momentum=0.9, decay=1e-5),
              loss='categorical_crossentropy',
              # metrics=['accuracy', CohenKappa(num_classes=4)]
              metrics=['accuracy',],)

model.fit(train_generator,
          # epochs=EPOCHS_WARMUP,
          epochs=50,
          batch_size=BATCH_SIZE,
          callbacks=callbacks_list,
          validation_data=val_generator,)

print('### END WARMUP ###')

from tensorflow_addons.metrics import CohenKappa
from keras.models import load_model

model = load_model(MODEL_PATH)

for layer in model.layers:
    layer.trainable = True

model.compile(# optimizer=Adam(1e-3),
              optimizer=Adam(LEARNING_RATE),
              # optimizer=RectifiedAdam(total_steps=10000, warmup_proportion=0.1, min_lr=1e-5),
              # optimizer=Adagrad(lr=LEARNING_RATE),
              # optimizer=SGD(lr=1e-3, momentum=0.9, decay=1e-5),
              # optimizer=SGD(lr=0.01, momentum=0.9, decay=0.001),
              # optimizer=SGD(lr=1e-5, momentum=0.9, decay=1e-6),
              loss='categorical_crossentropy',
              # loss=WeightedKappaLoss(num_classes=4),
              # metrics=['accuracy', CohenKappa(num_classes=4)]
              metrics=['accuracy',],)

start = time.time()

history = model.fit(train_generator, 
                    # epochs=EPOCHS, 
                    epochs=50,
                    callbacks=callbacks_list, 
                    batch_size=BATCH_SIZE,
                    validation_data=val_generator)

elapsed_time = time.time() - start
print('Elapsed time: {:.2f}[hours]'.format(elapsed_time / 3600))

### START WARMUP ###
Epoch 1/50
792/792 [==============================] - 148s 164ms/step - loss: 2.2916 - accuracy: 0.2597 - val_loss: 1.3515 - val_accuracy: 0.3970
Accuracy: 0.397, QWK: 0.000
Epoch 2/50
792/792 [==============================] - 128s 161ms/step - loss: 1.4211 - accuracy: 0.3381 - val_loss: 1.3677 - val_accuracy: 0.3970
Accuracy: 0.397, QWK: 0.000
Epoch 3/50
792/792 [==============================] - 127s 160ms/step - loss: 1.3629 - accuracy: 0.3601 - val_loss: 1.3608 - val_accuracy: 0.3970
Accuracy: 0.397, QWK: 0.000
Epoch 4/50
792/792 [==============================] - 128s 161ms/step - loss: 1.3727 - accuracy: 0.3807 - val_loss: 1.4002 - val_accuracy: 0.1709

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Accuracy: 0.171, QWK: 0.000
Epoch 5/50
792/792 [==============================] - 127s 161ms/step - loss: 1.3802 - accuracy: 0.3449 - val_loss: 1.3450 - val_accuracy: 0.3970
Accuracy: 0.397, QWK: 0.000
Epoch 6/50
792/792 [========

# evaluate

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
# cohen_kappa = history.history['cohen_kappa']
# val_cohen_kappa = history.history['val_cohen_kappa']
epochs = range(1, len(acc) + 1)

fig = plt.figure(0)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'b', color='orange', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

fig.savefig('acc.jpg')

plt.show()

fig = plt.figure(1)

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'b', color='orange', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

fig.savefig('loss.jpg')

plt.show()

# fig = plt.figure(2)

# plt.plot(epochs, cohen_kappa, 'b', label='Training cohen kappa')
# plt.plot(epochs, val_cohen_kappa, 'b', color='orange', label='Validation cohen kappa')
# plt.title('Training and validation cohen kappa')
# plt.legend()

# fig.savefig('cohen_kappa.jpg')

# plt.show()

In [ ]:
from tensorflow_addons.metrics import CohenKappa
from keras.models import load_model

clf = load_model(MODEL_PATH)
# clf = load_model(MODEL_PATH, custom_objects={'cohen_kappa': CohenKappa(num_classes=4)})

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix 
import matplotlib.pyplot as plt
import seaborn as sns

predicted = np.argmax(clf.predict(test_generator), axis=1)
test_labels = test_generator.classes

cf_matrix = confusion_matrix(test_labels, predicted)

fig = plt.figure(figsize=(16, 8))

sns.heatmap(cf_matrix, annot=True, cmap='Greens', xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES)

plt.xlabel('Predicted')
plt.ylabel('Actual')

fig.savefig('cf_matrix.jpg')

plt.show()

In [ ]:
from sklearn.metrics import cohen_kappa_score, accuracy_score

print('Test cohen kappa score: %.3f' % cohen_kappa_score(np.argmax(clf.predict(test_generator), axis=1), test_generator.classes, weights='quadratic'))
print('Test accuracy score : %.3f' % accuracy_score(np.argmax(clf.predict(test_generator), axis=1), test_generator.classes))

# result

|model|optimizer warmup|optimizer|test cohen kappa|test accuracy|Public Leaderbord|
|:--:|:--:|:--:|:--:|:--:|:--:|
|ResNet50|Adam(1e-3)|Adam(1e-4)|1.000|1.000|0.8934451|
|ResNet50|SGD(lr=0.2, momentum=0.9, decay=0.01)|SGD(lr=0.01, momentum=0.9, decay=0.001)|0.000|0.279|FFFF|
|EfficientNetB5|Adam(1e-3)|Adam(1e-4)|1.000|1.000|0.9471134|
|EfficientNetB7|Adam(1e-3)|Adam(1e-4)|1.000|1.000|FFFF|
|FFFF|FFFF|FFFF|FFFF|FFFF|FFFF|

# submit

In [ ]:
import numpy as np
from PIL import Image

preds = []

for image_id in df_preds['id']:
    image = Image.open(TEST_IMAGE_PATH+'/'+image_id)
    # image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    image = np.expand_dims(image, axis=0)
    image = image / 255.
    preds.append(np.argmax(clf.predict(image)))

df_preds['label'] = preds

df_preds.to_csv('/content/drive/MyDrive/SANDBOX/DATASET/submission.csv', index=False, header=False)

df_preds.head()

# Colab

```javascript
function ClickConnect(){ 
console.log("Working"); 
document.querySelector("#comments > span").click()
}
setInterval(ClickConnect,500000)
```